In [4]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [49]:
df = pd.read_csv('WELFake_Dataset.csv')
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [51]:
df['label'].value_counts()

1    37106
0    35028
Name: label, dtype: int64

In [52]:
df.shape

(72134, 4)

In [53]:
df.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [54]:
df = df.dropna()

In [55]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [56]:
df.shape

(71537, 4)

In [57]:
df.reset_index(inplace = True)
df.head()

,index,Unnamed: 0,title,text,label
0,0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
2,3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
3,4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
4,5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [58]:
#For time constraint we will only work on the title and not the text column
df = df.drop(['Unnamed: 0', 'text'], axis = 1)
df.head()

,index,title,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
3,4,SATAN 2: Russia unvelis an image of its terrif...,1
4,5,About Time! Christian Group Sues Amazon and SP...,1


In [19]:
#Date Preprocessing
#1. Tokenization
sample_data = 'The quick brown fox jumps over the lazy dog'
sample_data.split()

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

In [21]:
#make lowercase
[data.lower() for data in sample_data.split()]

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

In [23]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priyanuj/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
#Remove stopwords
stopwords = stopwords.words('english')
print(stopwords[0:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [27]:
[data for data in sample_data.lower().split() if data not in stopwords]

['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog']

In [28]:
# Stemming
ps = PorterStemmer()
sample_data_stemming = [ps.stem(data) for data in sample_data.lower().split() if data not in stopwords]
print(sample_data_stemming)

['quick', 'brown', 'fox', 'jump', 'lazi', 'dog']


In [30]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/priyanuj/nltk_data...


True

In [31]:
# Lemmatization
lm = WordNetLemmatizer()
sample_data_lemma = [lm.lemmatize(data) for data in sample_data.lower().split() if data not in stopwords]
sample_data_lemma

['quick', 'brown', 'fox', 'jump', 'lazy', 'dog']

In [59]:
df.head()

,index,title,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
2,3,"Bobby Jindal, raised Hindu, uses story of Chri...",0
3,4,SATAN 2: Russia unvelis an image of its terrif...,1
4,5,About Time! Christian Group Sues Amazon and SP...,1


In [60]:
#removing special characters from the texts
df['title'] = df['title'].str.replace('\W', ' ', regex = True)
df.head()

,index,title,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,2,UNBELIEVABLE OBAMA S ATTORNEY GENERAL SAYS MO...,1
2,3,Bobby Jindal raised Hindu uses story of Chri...,0
3,4,SATAN 2 Russia unvelis an image of its terrif...,1
4,5,About Time Christian Group Sues Amazon and SP...,1


In [61]:
###########
lm = WordNetLemmatizer()
corpus = []
for i in range(len(df)):
    review = re.sub('^a-zA-Z0-9', ' ', df['title'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(x) for x in review if x not in stopwords]
    review = " ".join(review)
    corpus.append(review)

In [62]:
len(corpus), df.shape[0]

(71537, 71537)

In [63]:
df['title'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9 11By  BlackLivesMatter And  FYF911 Terrorists  VIDEO '

In [64]:
corpus[0]

'law enforcement high alert following threat cop white 9 11by blacklivesmatter fyf911 terrorist video'

In [65]:
#Vectorization
tf = TfidfVectorizer()
x = tf.fit_transform(corpus).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [66]:
y = df['label']
y.head()

0    1
1    1
2    0
3    1
4    1
Name: label, dtype: int64

In [67]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 10, stratify = y)

In [68]:
len(X_train), len(X_test), len(y_train), len(y_test)

(50075, 21462, 50075, 21462)

In [ ]:
#Model Building
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

In [ ]:
y_pred = rf_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
#additionally from the video you can prepare the class for the whole project  